# Open SQL Connections

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os

#Define SQL Connections
sql_conn1 = pyodbc.connect('Trusted_Connection=Yes;'
                          'driver={SQL Server Native Client 11.0};'
                          'server=PWDA2SUPPSQL01\supinst01;'
                         'Database=PROD')


sql_conn2 = pyodbc.connect('Trusted_Connection=Yes;'
                          'driver={SQL Server Native Client 11.0};'
                          'server=pwaa11sqlsobi01;'
                         'Database=so_orders')

# Define Variables

In [2]:
i =-1

In [3]:
j = i*7-6

In [4]:
k = i*7

# Delete oldest weeks data

In [ ]:
DeleteWeekCode = """SELECT DISTINCT Datekey,YearWeek FROM [PROD].[dbo].[DimDate] WHERE WeekOffset = """ + str(i-8)
DeleteWeekdf = pd.read_sql_query(DeleteWeekCode,sql_conn1)
DeleteWeekno = DeleteWeekdf['YearWeek'].unique().tolist()

In [ ]:
#DC Fallback Update
df1 = pd.read_csv("""P:\Commercial Supply\SOM Analysis\Insight\Working documents\Sian\RFC\DC Fallback Data New (New Good DC).csv""")
df1New = df1[df1.Week != DeleteWeekno]
path1 = """P:\Commercial Supply\SOM Analysis\Insight\Working documents\Sian\RFC\DC Fallback Data New (New Good DC).csv"""
os.remove("P:\Commercial Supply\SOM Analysis\Insight\Working documents\Sian\RFC\DC Fallback Data New (New Good DC).csv")
df1New.to_csv(path1,mode='a',header=True,index = False)

# Define Stores DB SQL Query Text

In [ ]:
#Stores Query Text
Stores = """SELECT transaction_date_key datekey
,[product_number]
,[collection_location_number]
,[fulfilment_location_number]
,case when [proposition] = 'Store in Store' then 'Store in Store' when [proposition] = 'Micro' then 'Store in Store'
when [proposition] = 'Collection Point' then 'Collection Point' when [proposition] = 'Staff Collection Point' then 'Collection Point'
when ful.[location_type] = 'Standalone' then 'Spoke'
when [collection_location_number] in (4168,4226) then 'Store in Store'
else ful.[location_type] end as storetype
,sum([original_quantity_ordered]) SumQtyOrdered
,coalesce(ful.[parent_location],[collection_location_number]) HubLocn
,case
when ful.parent_location in (4716,4717,4719,4370) then collection_location_number
else ful.parent_location
end as 'HubLocn2'
,case
when ful.parent_location in (4716,4717,4719,4370)
then ful.parent_location
when ful2.parent_location in (4716,4717,4719,4370)
then ful2.parent_location else 'NA'
end as 'RFCNo'
,case
when ful.parent_location in (4716,4717,4719,4370)
then 'RFC Direct'
when ful2.parent_location in (4716,4717,4719,4370)
then 'RFC X-dock' else 'Non-RFC'
end as 'RFCStatus'
,case
when fulfilment_location_number in (4716,4717,4719,4370) then 'RFC Fulfilled'
else 'DC Fulfilled'
end as 'Fulfilled By'
,ser.Range
FROM [so_orders].[hddw].[store_orders] as ord
inner join [so_common].[dim].[date] as dat on ord.transaction_date_key=dat.Date_Key
left join [so_sales].[midas].[trading_hierarchy] as ser on ord.series_id=ser.series_number
left join [so_common].[location].[fulfilment] as ful on ord.collection_location_number=ful.location_number and (ord.transaction_date_key>=ful.start_date_key and ord.transaction_date_key<coalesce(ful.end_date_key,20990101))
left join [so_common].[location].[locations] loc on loc.location_number = ful.location_number
left join [so_common].[location].[proposition] prop on loc.[proposition_id]=prop.id
left join [so_common].[location].[fulfilment] as ful2 on ful.parent_location=ful2.location_number and (ord.transaction_date_key>=ful2.start_date_key and ord.transaction_date_key<coalesce(ful2.end_date_key,20990101))
where ord.fulfilment_location_number in (605,607,610,612,615,616,631,4370,4716,4717,4719)
and ord.status_code_key in (select id_key from [hddw].[order_status_key] where description ='Order Placed' or  description ='Stock Reserved')
and (ful.parent_location in (4716,4717,4719,4370) or ful2.parent_location in (4716,4717,4719,4370))
and convert(varchar(20), date_key,101) BETWEEN
(SELECT
DATEADD(DAY,""" + str(j) + """,cal_date)  from [so_common].[dim].[date]
where argos_week in (select argos_week from [so_common].[dim].[date]  where cal_date = convert(varchar(20),getdate(),101))
and financial_year in (select financial_year from [so_common].[dim].[date]  where cal_date = convert(varchar(20),getdate(),101))
AND day_of_week=7)
AND
(SELECT
DATEADD(DAY,""" + str(k) + """,cal_date)  from [so_common].[dim].[date]
where argos_week in (select argos_week from [so_common].[dim].[date]  where cal_date = convert(varchar(20),getdate(),101))
and financial_year in (select financial_year from [so_common].[dim].[date]  where cal_date = convert(varchar(20),getdate(),101))
AND day_of_week=7)
group by transaction_date_key
,[product_number]
,[collection_location_number]
,[fulfilment_location_number]
,case when [proposition] = 'Store in Store' then 'Store in Store' when [proposition] = 'Micro' then 'Store in Store'
when [proposition] = 'Collection Point' then 'Collection Point' when [proposition] = 'Staff Collection Point' then 'Collection Point'
when ful.[location_type] = 'Standalone' then 'Spoke'
when [collection_location_number] in (4168,4226) then 'Store in Store'
else ful.[location_type] end
,coalesce(ful.[parent_location],[collection_location_number])
,case
when ful.parent_location in (4716,4717,4719,4370) then collection_location_number
else ful.parent_location
end
,case
when ful.parent_location in (4716,4717,4719,4370)
then 'RFC Direct'
when ful2.parent_location in (4716,4717,4719,4370)
then 'RFC X-dock' else 'Non-RFC'
end
,case
when ful.parent_location in (4716,4717,4719,4370)
then ful.parent_location
when ful2.parent_location in (4716,4717,4719,4370)
then ful2.parent_location else 'NA'
end
,case
when fulfilment_location_number in (4716,4717,4719,4370) then 'RFC Fulfilled'
else 'DC Fulfilled'
end
,ser.Range"""

# Run Stores SQL Query

In [ ]:
Storesdf = pd.read_sql_query(Stores,sql_conn2)
Storesdf['HubLocn2'] = Storesdf['HubLocn2'].fillna(Storesdf['HubLocn'])

Create date list for weekoffset = i

In [ ]:
WeekCode = """SELECT DISTINCT Datekey,YearWeek FROM [PROD].[dbo].[DimDate] WHERE WeekOffset = """ + str(i)
Weekdf = pd.read_sql_query(WeekCode,sql_conn1)

DateList = ",".join([str(x) for x in Weekdf['Datekey'].unique().tolist()])
Weekno = Weekdf['YearWeek'].unique().astype(int)
Dates = Weekdf['Datekey'].unique().tolist()

In [ ]:
Storesdf.HubLocn2 = Storesdf.HubLocn2.astype(int)
Storesdf['product_number'] = Storesdf['product_number'].astype(int)

In [ ]:
from datetime import datetime, timedelta
x = str(min(Dates))
y = str(max(Dates))
        
startdate = datetime(year=int(x[0:4]), month=int(x[4:6]), day=int(x[6:8]))
enddate = datetime(year=int(y[0:4]), month=int(y[4:6]), day=int(y[6:8]))        
mindate = startdate - timedelta(days=3)

s = mindate.strftime("%Y%m%d")
t = enddate.strftime("%Y%m%d")

# Define SA SQL Query

In [ ]:
Unlock = """set transaction isolation level read uncommitted
SELECT loc.StoreNo,Catno,max(convert(int,RUTLUse)) Planned,max(convert(int,[Unlock])) Unlock,loc2.SterlingLocn as DC, OnCostSheet
FROM [PROD].[dbo].[FactAvailabilityImmediate] AS FAI
inner join [PROD].[dbo].[DimProduct_SA] as dpm on fai.ProductID=dpm.ProductID
inner join [PROD].[dbo].[DimLocation] as loc on fai.LocationID=loc.LocationID
inner join [PROD].[dbo].[DimLocation] as loc2 on fai.DCLocID=loc2.LocationID
WHERE TeirID=1 AND datekey in (""" + DateList + """)
GROUP BY loc.StoreNo,Catno,loc2.SterlingLocn, OnCostSheet"""

In [ ]:
GoodDCStock = """set transaction isolation level read uncommitted
SELECT datekey,Catno 'product_number'
,SterlingLocn RFCNo
,StockStatus as 'Good DC Stock'
from 
[SANDBOX].[PM].[dcavailb] GDS
inner join  [PROD].[dbo].[DimProduct_SA] PSA With(NoLock) on psa.ProductID=GDS.ProductID
inner join [PROD].[dbo].DimLocation LOC With(NoLock) on LOC.LocationID=GDS.StoreLocID
where datekey in (""" + DateList + """)
and SterlingLocn in ('4716','4717','4719','4370')"""

In [ ]:
GoodDC = pd.read_sql_query(GoodDCStock,sql_conn1)
GoodDC.RFCNo = GoodDC.RFCNo.astype(int)

In [ ]:
Unlockdf = pd.read_sql_query(Unlock,sql_conn1)
Unlockdf.DC = Unlockdf.DC.astype(int)
Unlockdf.columns = ['HubLocn2','product_number','Planned','Unlock','DC','OnCostSheet']

# Merge datasets and replace NAs

In [ ]:
merge_df1 = pd.merge(Storesdf,Unlockdf[['HubLocn2','product_number','Planned','Unlock','OnCostSheet']], how='left',on=('HubLocn2','product_number'))
merge_df1['Planned'] = merge_df1['Planned'].fillna(0)
merge_df1['Unlock'] = merge_df1['Unlock'].fillna(0)
merge_df1['OnCostSheet'] = merge_df1['OnCostSheet'].fillna(False)

In [ ]:
merge_df2 = pd.merge(merge_df1,GoodDC, how = 'left', on = ('product_number', 'RFCNo','datekey'))
merge_df2['Good DC Stock'] = merge_df2['Good DC Stock'].fillna(2)

In [ ]:
merge_df2sum = merge_df2.groupby(['product_number','collection_location_number','fulfilment_location_number','storetype','HubLocn','HubLocn2','RFCNo','RFCStatus','Fulfilled By','Range','Planned','Unlock','OnCostSheet','Good DC Stock']).agg(
    {'SumQtyOrdered' : 'sum'})

# Create final data set for export

In [ ]:
merge_df2sum['Week'] = int(Weekno)

In [ ]:
path3 = """P:\Commercial Supply\SOM Analysis\Insight\Working documents\Sian\RFC\DC Fallback Data New (New Good DC).csv"""
merge_df2sum.to_csv(path3,mode='a',header=False,index = True)

Close SQL Connection

In [ ]:
csr1 = sql_conn1.cursor()  
csr1.close()
del csr1
sql_conn1.close()

crsr2 = sql_conn2.cursor()
crsr2.close()
del crsr2
sql_conn2.close()